In [1]:
import pandas as pd
import numpy as np
import folium 
from shapely.geometry import Point
import geopandas as gpd
from cartiflette import carti_download

In [2]:
# =================================================================
# 1. CHARGEMENT DES DÉPARTEMENTS ET PRÉPARATION
# =================================================================

# Chargement des départements
departements = carti_download(
    values=["France"],
    crs=4326,
    borders="DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Filtrage France métropolitaine
departements = departements.loc[departements['INSEE_DEP'] < '970'] 
departements_proj = departements.to_crs(epsg=2154)  # Reprojection en Lambert-93


In [3]:
import pandas as pd
import os
import sys

# Définition du nom de fichier que nous avons créé précédemment
NOM_FICHIER_DESIRE = "BDD_Temp_Puy_de_Dome_2000_2007.csv"
chemin_complet = os.path.join(os.getcwd(), NOM_FICHIER_DESIRE)

# --- 1. Importer le fichier CSV ---
try:
    # Lecture du fichier CSV
    # Nous utilisons 'index_col=0' car l'exportation précédente (index=True) 
    # a généralement transformé l'ancien MultiIndex (time, lat, lon) en première colonne.
    df_puy_de_dome = pd.read_csv(chemin_complet, index_col=0) 
    print(f"✅ Fichier '{NOM_FICHIER_DESIRE}' importé avec succès dans 'df_puy_de_dome'.")

except FileNotFoundError:
    print(f"❌ Erreur : Le fichier CSV est introuvable à l'emplacement : {chemin_complet}")
    print("Veuillez vous assurer que le script d'exportation a été exécuté correctement.")
    sys.exit(1)
except Exception as e:
    print(f"❌ Erreur lors de l'importation du CSV : {e}")
    sys.exit(1)


✅ Fichier 'BDD_Temp_Puy_de_Dome_2000_2007.csv' importé avec succès dans 'df_puy_de_dome'.


In [4]:
df_puy_de_dome.head(10)

,latitude,longitude,number,expver,d2m,t2m,skt,stl1,stl2,stl3,stl4
valid_time,,,,,,,,,,,
2000-01-01,45.7,3.1,0,1,271.61633,273.55457,271.85330,273.67154,274.22733,276.09286,279.96063
2000-01-01,45.7,3.2,0,1,271.88000,273.81042,271.98610,273.89420,274.45780,276.35850,280.27900
2000-01-01,45.6,3.1,0,1,271.44446,273.26940,271.54470,273.80630,274.33865,276.16122,279.95282
2000-01-01,45.6,3.2,0,1,271.72570,273.57800,271.67752,273.89030,274.43630,276.29990,280.18915
2000-02-01,45.7,3.1,0,1,274.20380,277.49805,276.83124,276.67590,276.50793,276.44196,278.00046
2000-02-01,45.7,3.2,0,1,274.52216,277.78516,276.97968,276.89465,276.74622,276.70172,278.27390
2000-02-01,45.6,3.1,0,1,274.05340,277.28125,276.58905,276.53723,276.43762,276.46150,278.04538
2000-02-01,45.6,3.2,0,1,274.38740,277.60742,276.74920,276.70715,276.60754,276.62750,278.21530
2000-03-01,45.7,3.1,0,1,273.84520,278.00778,277.79504,278.04703,278.01770,277.86444,277.65216


In [5]:
# Cr"ation du dataframe geopandas
gdf_geo_puy_de_dome = gpd.GeoDataFrame(
    df_puy_de_dome,
    geometry=gpd.points_from_xy(df_puy_de_dome['longitude'], df_puy_de_dome['latitude']),
    crs="EPSG:4326"
)
gdf_geo_puy_de_dome = gdf_geo_puy_de_dome.to_crs(departements_proj.crs)

In [6]:
gdf_geo_puy_de_dome.index

Index(['2000-01-01', '2000-01-01', '2000-01-01', '2000-01-01', '2000-02-01',
       '2000-02-01', '2000-02-01', '2000-02-01', '2000-03-01', '2000-03-01',
       ...
       '2007-10-01', '2007-10-01', '2007-11-01', '2007-11-01', '2007-11-01',
       '2007-11-01', '2007-12-01', '2007-12-01', '2007-12-01', '2007-12-01'],
      dtype='object', name='valid_time', length=384)

In [10]:
gdf_geo_puy_de_dome.columns

Index(['latitude', 'longitude', 'number', 'expver', 'd2m', 't2m', 'skt',
       'stl1', 'stl2', 'stl3', 'stl4', 'geometry'],
      dtype='object')

In [46]:
# On viens prendre les données d'une année en particulier, par exemple 2000 et 2001
gdf_geo_puy_de_dome.index = pd.to_datetime(gdf_geo_puy_de_dome.index)
gdf_2000 = gdf_geo_puy_de_dome[gdf_geo_puy_de_dome.index.year == 2000]
gdf_geo_puy_de_dome.index = pd.to_datetime(gdf_geo_puy_de_dome.index)
gdf_2007 = gdf_geo_puy_de_dome[gdf_geo_puy_de_dome.index.year == 2007]

gdf_2000.index = gdf_2000.index.strftime("%d/%m")
gdf_2007.index = gdf_2007.index.strftime("%d/%m")

In [60]:
# calcule de la différence entre les 2 années en conservant la latitude et longitude associées
df_diff = gdf_2000[["longitude", "latitude","geometry"]].copy()
df_diff["d2m_diff"] = gdf_2000["d2m"] - gdf_2007["d2m"]
gdf_geo_puy_de_dome = gpd.GeoDataFrame(
    df_diff,
    geometry=gpd.points_from_xy(df_diff['longitude'], df_diff['latitude']),
    crs="EPSG:4326"
)

In [61]:
df_diff.head()

,longitude,latitude,geometry,d2m_diff
valid_time,,,,
01/01,3.1,45.7,POINT (707781.484 6511163.615),-3.00677
01/01,3.2,45.7,POINT (715562.955 6511178.397),-3.04194
01/01,3.1,45.6,POINT (707795.548 6500058.454),-2.95599
01/01,3.2,45.6,POINT (715591.083 6500073.263),-2.98140
01/02,3.1,45.7,POINT (707781.484 6511163.615),-0.63134


In [ ]:
#FILTRAGE SPATIAL SUR LE PUY-DE-DÔME
departement_63 = departements_proj[departements_proj['INSEE_DEP'] == '63'].copy()

points_diff_63 = gpd.sjoin(
    df_diff, 
    departement_63[['INSEE_DEP', 'geometry']],
    predicate="within"
)

points_diff_63 = points_diff_63.to_crs(epsg=4326)
departement_63 = departement_63.to_crs(epsg=4326)

In [ ]:
points_diff_63.head()

,longitude,latitude,geometry,d2m_diff,index_right,INSEE_DEP
valid_time,,,,,,
01/01,3.1,45.7,POINT (3.1 45.7),-3.00677,18,63
01/01,3.2,45.7,POINT (3.2 45.7),-3.04194,18,63
01/01,3.1,45.6,POINT (3.1 45.6),-2.95599,18,63
01/01,3.2,45.6,POINT (3.2 45.6),-2.98140,18,63
01/02,3.1,45.7,POINT (3.1 45.7),-0.63134,18,63


In [ ]:
# =================================================================
# 4. CALCUL DU CENTRE POUR LA CARTE
# =================================================================

bounds = departement_63.total_bounds  # [minx, miny, maxx, maxy]
CENTER_LON = (bounds[0] + bounds[2]) / 2
CENTER_LAT = (bounds[1] + bounds[3]) / 2


In [ ]:
import branca.colormap as cm



# Création d'un colormap 
colormap = cm.linear.RdYlBu_11.scale(
    df_diff["d2m_diff"].min(),
    df_diff["d2m_diff"].max()
)
# Normaliser les valeurs
min_val = df_diff["d2m_diff"].min()
max_val = df_diff["d2m_diff"].max()


for dates in points_diff_63.index:
    m = folium.Map(
            location=[CENTER_LAT, CENTER_LON], 
            zoom_start=9, 
            tiles="CartoDB positron"
            )
    # Bordure du département
    folium.GeoJson(
        departement_63,
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0
        },
        name="Puy-de-Dôme (63)"
    ).add_to(m)
    colormap.add_to(m)
    for i in range(len(points_diff_63.loc[dates, "d2m_diff"])):
        val = points_diff_63.loc[dates, "d2m_diff"][i]
        lat = points_diff_63.loc[dates, "latitude"][i]
        lon = points_diff_63.loc[dates, "longitude"][i]
        
        color = colormap(val- min_val / (max_val - min_val))
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8
        ).add_to(m)
        
    dates_formatted = dates.replace("/", ".")
    m.save(f"carte_date_t/carte_puy_de_dome_difference{dates_formatted}.html")






# m.save("carte_date_t/carte_puy_de_dome_difference.html")
# print("\nCarte sauvegardée : carte_puy_de_dome_difference_amas.html")

#m

C:\Users\loicd\AppData\Local\Temp\ipykernel_19532\1016109666.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  val = points_diff_63.loc[dates, "d2m_diff"][i]
C:\Users\loicd\AppData\Local\Temp\ipykernel_19532\1016109666.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat = points_diff_63.loc[dates, "latitude"][i]
C:\Users\loicd\AppData\Local\Temp\ipykernel_19532\1016109666.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  